In [3]:
import math
import sys


'''readFileString() returns the values of the sudoku as a string...used to make a grid_dictionary'''
def readFileString():
    print("Enter the path of the file")
    p = input()
    f = open(p)
    #f = open("E:\\waterloo_documents\\AI\\SudokuProblem\\problems\\1\\1.sd")
    var = f.read()
    var = var.replace('\n','')
    var = var.replace(" ","")
    #print(var)
    return var
            
            
'''This function will take two iterables and returns the cartesian product of them as a list'''
def cartesian_product(x,y):
    return [a+b for a in x for b in y]

'''eliminate() function takes as input the dictionary of positions and values as input.
It returns a dictionary which gives the possible values any key can take'''

def eliminate(grid_dict):
    for k,v in grid_dict.items():
        if len(v) != 1: # if the box needs elimination
            peers = peer_dict[k]    # get all the peers
            peer_values = set([grid_dict[p] for p in peers if len(grid_dict[p]) == 1])
            grid_dict[k] = ''.join(set(grid_dict[k]) - peer_values)
    return grid_dict


'''This function gives labels to the rows and cols. It then returns a peer_dict which stores the list of peers
(peers are those positions which cannot have the same value) for each position and a grid_dict where the 
cartesian product of the labels are the keys and the number at this position is the value in the dictionary.'''

def initializing():
    start = readFileString()
    rows = 'ABCDEFGHI'
    cols = '123456789'
    boxes = cartesian_product(rows, cols)
    row_units = [cartesian_product(r, cols) for r in rows]
    col_units = [cartesian_product(rows, c) for c in cols]
    box_units = [cartesian_product(r,c) 
                        for r in ['ABC', 'DEF', 'GHI'] 
                        for c in ['123','456','789']]

    unit_list = row_units + col_units + box_units
    unit_dict = dict((box, [unit for unit in unit_list if box in unit]) for box in boxes)
    peer_dict = dict((box, set(sum(unit_dict[box], [])) - set([box])) for box in boxes)
    grid_dict = dict(zip(boxes, start))
    return peer_dict, grid_dict

'''find_empty_location takes as input:
1) grid_dict: It is a dictionary of positions and their values
2) temp: the value of temp=[0,0]

The function finds a position in arr having value 0 and stores its position in temp '''
        
def find_empty_locnew(grid_dict, empty_pos):
    for k, v in grid_dict.items():
        if(v=='0'):
            empty_pos = k
            return empty_pos
        
'''get_degree function returns the number of peers a given position constraints. It returns a variable degree that gives
the number of constraints a position k has'''
def get_degree(k,v,peer_dict,grid_dict):
    degree = 0
    #print("k",k)
    peer_list = peer_dict[k]
    #print("peers",peer_list)
    for peer in peer_list:
        #print("peer",peer)
        #print("value",grid_dict[peer])
        if(grid_dict[peer]=='0'):
            #print("inside loop")
            degree = degree + 1
    #print("degree",degree)
    return degree


'''This function takes as input:
1) peer_dict: the dictionary having a position as key and its peers as value
2) remaining_values: the dictionary returned by the eliminate() function. It contains all the possible values as position can have
3) value: the value which is selected for a given position
4) key: the position(key) in the grid_dict

The function checks whether any peers of the current key has only the above value as an option. 
If so it returns false, otherwise returns true'''

def forward_checknew(peer_dict, remaining_values, value, key):
    peer_list_key = peer_dict[key]
    for k in peer_list_key:
        x = remaining_values[k]
        if len(x) == 0:
            return 0
    return 1
    

'''Solves the sudoku using forward checking and backtracking'''
def sudoku_forwardchecking(peer_dictionary, grid_dictionary_param):
    global COUNTER
    empty_pos = ""
    grid_dictionary = dict(grid_dictionary_param)
    grid_dict_original = dict(grid_dictionary)
    
    '''finds a location which is empty and stores it in selected_key variable'''
    selected_key = find_empty_locnew(grid_dict_original, empty_pos)
    
            
    '''If there is not empty position left, then that means that the sudoke is solved. So it prints the output and returns true'''    
    if not selected_key:
        print("sudoku using heuristics")
        print(grid_dict_original)
        print("steps are:",COUNTER)
        return True
   
    '''Replaces the 0's by 123456789'''
    for k,v in grid_dictionary.items():
        if v == '0':
            grid_dictionary[k] = '123456789'
    
    '''stores the possible values in every position in remaining_values_dictionary'''
    remaining_values_dictionary = eliminate(grid_dictionary)
    
    '''Finds a key which has the least number of remaining values'''
    max_length = 9
    for k, v in remaining_values_dictionary.items():
        if(len(v)!=1):
            if(len(v)<max_length):
                val_max_length = v
                selected_key = k
                max_length = len(v)
                #print("got smaller length")
    
    mrv_dict = {}
    '''stores the positions having the least number of remaining values in the dictionary mrv_dict()'''
    for k, v in remaining_values_dictionary.items():
        if(len(v)==max_length):
            mrv_dict[k] = v
                
    '''If mrv_dict has more than one entries, then the position which is the most constraining is selected.'''
    if len(mrv_dict) == 1:
        selected_key = list(mrv_dict.keys())[0]
    else:
        degree_dict = {}
        for k, v in mrv_dict.items():
            degree = get_degree(k,v,peer_dictionary,grid_dict_original)
            degree_dict[k] = degree
            
        
        max_val = 0
        for k, v in degree_dict.items():
            if(v>max_val):
                max_val = v
                selected_key = k
    
    
    '''creates a list which stores the possible values of the selected_key'''
    values = list(remaining_values_dictionary[selected_key])
    
    '''selects any value from the list, performs forward check and if it there are no clashes, then updates the dictionary 
    and calls the function again. If forward checking fails and there are no possible values available, then it backtracks'''
    while len(values) != 0:   
        value = values[0]
        values.remove(value)
        
        if forward_checknew(peer_dictionary, remaining_values_dictionary, value, selected_key ):
            #print("forward checking successful")
            grid_dictionary[selected_key] = value
            grid_dict_original[selected_key] = value
            
            '''counts the steps taken to reach the goal state and terminates if the number of steps exceed 10000'''
            COUNTER = COUNTER+1
            #print("steps",COUNTER)
            if(COUNTER>=10000):
                sys.exit("steps exceed 10000")

            '''call itself'''
            if(sudoku_forwardchecking(peer_dictionary, grid_dict_original)):
                return True
            else:
                grid_dictionary[selected_key] = '0'
                grid_dict_original[selected_key] = '0'
        #print("forward checking fails")
    
    '''this triggers backtracking'''
    #print("backtracking")
    return False

    
COUNTER = 0    
# Driver main function to test above functions 
if __name__=="__main__": 
    
    #steps = 0
    peer_dict, grid_dict = initializing()
    if(sudoku_forwardchecking(peer_dict, grid_dict)):
        print("Successful!")

Enter the path of the file
E:\waterloo_documents\AI\SudokuProblem\problems\12\4.sd
sudoku using heuristics
{'A1': '4', 'A2': '1', 'A3': '8', 'A4': '3', 'A5': '2', 'A6': '5', 'A7': '6', 'A8': '7', 'A9': '9', 'B1': '3', 'B2': '5', 'B3': '9', 'B4': '1', 'B5': '6', 'B6': '7', 'B7': '2', 'B8': '4', 'B9': '8', 'C1': '7', 'C2': '2', 'C3': '6', 'C4': '4', 'C5': '9', 'C6': '8', 'C7': '3', 'C8': '1', 'C9': '5', 'D1': '8', 'D2': '6', 'D3': '4', 'D4': '7', 'D5': '1', 'D6': '9', 'D7': '5', 'D8': '2', 'D9': '3', 'E1': '2', 'E2': '9', 'E3': '1', 'E4': '5', 'E5': '3', 'E6': '4', 'E7': '8', 'E8': '6', 'E9': '7', 'F1': '5', 'F2': '7', 'F3': '3', 'F4': '2', 'F5': '8', 'F6': '6', 'F7': '1', 'F8': '9', 'F9': '4', 'G1': '6', 'G2': '3', 'G3': '7', 'G4': '9', 'G5': '5', 'G6': '1', 'G7': '4', 'G8': '8', 'G9': '2', 'H1': '9', 'H2': '8', 'H3': '5', 'H4': '6', 'H5': '4', 'H6': '2', 'H7': '7', 'H8': '3', 'H9': '1', 'I1': '1', 'I2': '4', 'I3': '2', 'I4': '8', 'I5': '7', 'I6': '3', 'I7': '9', 'I8': '5', 'I9': '6'}
s